# 12 Explore kNN distances for other point distributions on the sphere

Following the same path as with the oil glands, how does the average distance between neighboring points look like if the points are distributed uniformly across a unit sphere surface?

- Various spherical distributions are simulated
    - Uniform
    - von Mises-Fisher (vMF)
    - Bingham
- For a fixed distribution, 161 point samples are simulated.
- Each sample has $N_k$ points, were $N_k$ is the number of individual oil glands of the $k$-th citrus.
- The average distance to the 25 nearest neighbors is computed
- The results are stored in the `merged` directory

<img src="https://www.egr.msu.edu/~amezqui3/citrus/figs/uniform_distribution_25knn.svg" width=700>

In [1]:
library(sphunif)
library(FNN)
library(Directional)

Loading required package: Rcpp


Attaching package: ‘Directional’


The following object is masked from ‘package:FNN’:

    knn.reg




### Set the appropriate directory

(I still don't know a nice way to tell R to set its working directory wherever the notebook is located, unlike python)

- Get the number of individual oil glands for each citrus

In [6]:
setwd('~/documents/vitaminC_morphology/data/allometry')
filename <- 'citrus_sizes.csv'
data <- read.csv(filename, header=T)

In [7]:
data

citrus_id,label,whole,exocarp,endocarp,rind,spine,mesocarp,oil_glands,num_glands
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
C01_CRC2317_18B-18-9,L00,275019544,58370453,150746200,81211503,11814268,69397235,391835,8217
C01_CRC2317_18B-18-9,L01,148889891,28406309,86537041,44943507,5992364,38951143,309626,5768
C01_CRC2317_18B-18-9,L02,141881396,28608241,87527253,35790295,2907487,32882808,294692,6005
C01_CRC2317_18B-18-9,L03,107342471,25489589,64476218,25670709,2785839,22884853,300443,5817
C02_CRC3919_18A-24-1,L00,194197677,43113850,113981920,49850355,3644418,46205937,1071689,13185
C02_CRC3919_18A-24-1,L01,154125194,26401120,93246923,46556136,2924908,43631228,1409217,12603
C02_CRC3919_18A-24-1,L02,148820067,30376176,97621471,31539222,1921415,29617807,1314165,12192
C03_CRC3593_18A-8-3,L00,221097244,26002141,144961718,66386257,5456364,60929893,337334,6596
C03_CRC3593_18A-8-3,L01,213522115,29759164,154430997,43538792,3769022,39769418,324382,6317


### Uniform distribution simulation

- Every equal-area patch has the same probability of containing a point

In [8]:
set.seed(42)
knn_max <- 25
KNN <- base::matrix(0, nrow=nrow(data), ncol=knn_max)

for(i in 1:nrow(data)){
    sph_unif <- sphunif::r_unif_sph(data$num_glands[i], 3, 1)
    dists <- FNN::knn.dist(sph_unif[,,1], k=knn_max)
    avgs <- base::colMeans(dists)
    KNN[i,] <- avgs
}

In [10]:
filename <- 'uniform_distribution_knn.csv'
MASS::write.matrix(KNN, filename, sep=',')

### Bingham distribution

- Resembles a girdle

In [12]:
KNN <- base::matrix(0, nrow=nrow(data), ncol=knn_max)
for(i in 1:nrow(data)){
    bingham <- Directional::f.rbing(data$num_glands[i], c(2,1), fast=TRUE)[[1]]
    dists <- FNN::knn.dist(bingham, k=knn_max)
    avgs <- base::colMeans(dists)
    KNN[i,] <- avgs
}

In [14]:
filename <- 'bingham_distribution_2,1_knn.csv'
MASS::write.matrix(KNN, filename, sep=',')

### von Mises-Fisher distribution

- The equivalent of the normal distribution on a sphere

In [18]:
KNN <- base::matrix(0, nrow=nrow(data), ncol=knn_max)

for(i in 1:nrow(data)){
    vmf <- Directional::rvmf(data$num_glands[i], c(0,0,1), k=1)
    dists <- FNN::knn.dist(vmf, k=knn_max)
    avgs <- base::colMeans(dists)
    KNN[i,] <- avgs
}

In [19]:
filename <- 'vmf_distribution_1_knn.csv'
MASS::write.matrix(KNN, filename, sep=',')

### Results will be plotted in a later notebook

# References

- **Mardia, KV**, **Jupp, PE** (1999) _Directional Statistics_. John Wiley & Sons, Ltd. [DOI: 10.1002/9780470316979](https://doi.org/10.1002/9780470316979)